In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from functions import basic

In [4]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython){
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

In [5]:
# Reading initial positions file and saving positions into a dataframe
filename = "fcc100a256.txt"
atoms_position = basic.read_file(filename)
atoms_position.head()

,x,y,z
0,2.0802,2.0802,0.0
1,0.0000,0.0000,0.0
2,0.0000,4.1604,0.0
3,2.0802,6.2406,0.0
4,0.0000,8.3208,0.0


### Test zone - used for quick tests and monkey patch

In [66]:
import time

In [65]:
rc = 4.625
rp = 4.375

In [68]:
# Reading initial positions file and saving positions into a dataframe
filename = "fcc100a256.txt"
atoms_position = basic.read_file(filename)
atoms_position.head()


def find_neighbours(atoms_position,rc):
    neighbours = []
    for index,row in tqdm(atoms_position.iterrows()):
        this_neighbours = []
        for index2,row2 in atoms_position.iterrows():
            if (index != index2) & (np.sqrt(np.sum((np.abs(row-row2)**2)))<rc):
                this_neighbours.append(index2)
        neighbours.append(this_neighbours)
    return neighbours

start = time.process_time()
neighbours = find_neighbours(atoms_position.head(10),rc)
print(time.process_time() - start)
#atoms_position['neighbours'] = neighbours
#atoms_position.head()

0it [00:00, ?it/s]

0.11216240699999958


In [70]:
# Reading initial positions file and saving positions into a dataframe
filename = "fcc100a256.txt"
atoms_position = basic.read_file(filename)
atoms_position.head()


def find_neighbours(atoms_position,rc):
    neighbours = []
    for ix,row in tqdm(enumerate(atoms_position.to_numpy())):
        this_neighbours = []
        for ix2,row2 in enumerate(atoms_position.to_numpy()):
            if (ix != ix2) & (np.sqrt(np.sum((np.abs(row-row2)**2)))<rc):
                this_neighbours.append(ix2)
        neighbours.append(this_neighbours)
    return neighbours

start = time.process_time()
neighbours = find_neighbours(atoms_position,rc)
print(time.process_time() - start)
#atoms_position['neighbours'] = neighbours
#print(time.process_time() - start)
#atoms_position.head()

0it [00:00, ?it/s]

0.779186885999998


### comparison with old version

In [72]:
def read_file(filename):
    x,y,z = np.genfromtxt(filename, unpack=True)
    sx = x[0]
    sy = y[0]
    sz = z[0]
    x = x[1:]
    y = y[1:]
    z = z[1:]
    return sx,sy,sz,x,y,z
    

def find_neighbours(x,y,z,sx,sy,sz,nat):
    fake_x = np.zeros(nat)
    fake_y = np.zeros(nat)
    fake_z = np.zeros(nat)
    coord_x =  np.column_stack((x,fake_y,fake_z))
    coord_y =  np.column_stack((fake_x,y,fake_z))
    coord_z =  np.column_stack((fake_x,fake_y,z))
    x_distances = np.sum(coord_x[:,None,:] - coord_x, axis=-1)
    y_distances = np.sum(coord_y[:,None,:] - coord_y, axis=-1)        
    z_distances = np.sum(coord_z[:,None,:] - coord_z, axis=-1)    
    #starting PBC on x and y coordinates
    #to remove PBC just comment the following lines and the similar ones in calc_force
    mask_x_plus = x_distances>(0.5*sx)
    mask_y_plus = y_distances>(0.5*sy)
    mask_x_minus = x_distances<(-0.5*sx)
    mask_y_minus = y_distances<(-0.5*sy)
    x_distances = x_distances - sx*mask_x_plus + sx*mask_x_minus
    y_distances = y_distances - sy*mask_y_plus + sy*mask_y_minus
    #ending PBC
    neighbours_distances = np.sqrt((x_distances**2 + y_distances**2 + z_distances**2))
    mask1 = neighbours_distances<rc
    mask2 = neighbours_distances>0
    mask = mask1 * mask2
    return mask,neighbours_distances

In [81]:
import time
start = time.process_time()
# your code here    

nat = 256
sx,sy,sz,x,y,z = read_file('fcc100a256.txt')
mask,neighbours_distances = find_neighbours(x,y,z,sx,sy,sz,nat)
print(time.process_time() - start)

0.011283995000006541


In [105]:
def read_file(filename):
    x,y,z = np.genfromtxt(filename, unpack=True)
    sx = x[0]
    sy = y[0]
    sz = z[0]
    x = x[1:]
    y = y[1:]
    z = z[1:]
    n_atoms = len(x)
    return n_atoms,sx,sy,sz,x,y,z
    

def find_neighbours(nat,sx,sy,sz,x,y,z,PBC=False):
    fake_x = np.zeros(nat)
    fake_y = np.zeros(nat)
    fake_z = np.zeros(nat)
    coord_x =  np.column_stack((x,fake_y,fake_z))
    coord_y =  np.column_stack((fake_x,y,fake_z))
    coord_z =  np.column_stack((fake_x,fake_y,z))
    x_distances = np.sum(coord_x[:,None,:] - coord_x, axis=-1)
    y_distances = np.sum(coord_y[:,None,:] - coord_y, axis=-1)        
    z_distances = np.sum(coord_z[:,None,:] - coord_z, axis=-1)    
    if PBC:
        mask_x_plus = x_distances>(0.5*sx)
        mask_y_plus = y_distances>(0.5*sy)
        mask_x_minus = x_distances<(-0.5*sx)
        mask_y_minus = y_distances<(-0.5*sy)
        x_distances = x_distances - sx*mask_x_plus + sx*mask_x_minus
        y_distances = y_distances - sy*mask_y_plus + sy*mask_y_minus
    neighbours_distances = np.sqrt((x_distances**2 + y_distances**2 + z_distances**2))
    mask1 = neighbours_distances<rc
    mask2 = neighbours_distances>0
    mask = mask1 * mask2
    return mask,neighbours_distances

In [106]:
start = time.process_time()
mask,neighbours_distances = find_neighbours(*read_file('fcc100a256.txt'),PBC=False)
print(time.process_time() - start)

0.022830318000004013
